In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pickle
import pandas as pd
import re
import spacy
from sklearn.metrics import accuracy_score


# não compartilhem a base de dados envolvidas nesse projeto

In [ ]:
# função de lematização completa do documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
        
    return ' '.join(sent)

# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)  
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZâêîôûÂÊÎÔÛáéíóúÁÉÍÓÚàèìòùÀÈÌÒÙãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto


In [ ]:
nlp = spacy.load('pt_core_news_sm')
novo_vetor = pickle.load(open('modelo_verorizacao.pkl', 'rb'))
novo_modelo = pickle.load(open('modelo_classificacao.pkl', 'rb'))

In [ ]:
# carrega os Tweets classificados
dfTweetsValida = pd.read_csv('dados/tweets_classificados_validacao.txt') #dfTweetsClass
dfTweetsValida.info()
dfTweetsValida.sentimento.value_counts()

In [ ]:
# aplica normalização no dataframe
dfTweetsValida['texto_trat'] = dfTweetsValida.texto.apply(fn_limpa_texto)

# aplica a lematização no dataframe
dfTweetsValida['texto_lemma_verb'] = dfTweetsValida.texto_trat.apply(fn_lematiza_texto)

# apaga tweets nulos
dfTweetsValida.dropna(inplace=True)

In [ ]:
text_vect = novo_vetor.transform(dfTweetsValida.texto_lemma_verb)

In [ ]:
predito = novo_modelo.predict(text_vect)

In [ ]:
predito

In [ ]:
# valida a parformance
accuracy = accuracy_score(predito, dfTweetsValida.sentimento)

print(text_vect.shape)
print(accuracy)